In [ ]:
pip install pyspark==3.2.1 systemds==2.2.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 3.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 MB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.0/199.0 kB 20.0 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853629 sha256=07fa87515c65dea38aff0e709faf2af4939827f4a26a5d08ce6083ff5c81608c
  Stored in directory: /root/.cache/pip/wheels/15/97/bd/52908574a60b5f8e3dc4dc5a0b5be8a59ac20986ee51c2611b
Successfully built pyspark
  Attempting uninstall: py4j
    Found existing installation: py4j 0.10.9.7
    Uninstalling py4j-0.10.9.7:
      Successfully uninstalled py4j-0.10.9.7


In [ ]:
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.config("ML Pipeline").getOrCreate()

In [ ]:
!wget https://github.com/IBM/coursera/blob/master/coursera_ml/shake.parquet?raw=true
!mv shake.parquet?raw=true shake.parquet

--2023-06-27 08:53:47--  https://github.com/IBM/coursera/blob/master/coursera_ml/shake.parquet?raw=true
Resolving github.com (github.com)... 192.30.255.112
Connecting to github.com (github.com)|192.30.255.112|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://github.com/claimed-framework/component-library/blob/master/coursera_ml/shake.parquet?raw=true [following]
--2023-06-27 08:53:47--  https://github.com/claimed-framework/component-library/blob/master/coursera_ml/shake.parquet?raw=true
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/claimed-framework/component-library/raw/master/coursera_ml/shake.parquet [following]
--2023-06-27 08:53:47--  https://github.com/claimed-framework/component-library/raw/master/coursera_ml/shake.parquet
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubuserco

In [ ]:
df=spark.read.parquet('shake.parquet')

In [ ]:
df.show()

+-----+---------+-----+-----+-----+
|CLASS| SENSORID|    X|    Y|    Z|
+-----+---------+-----+-----+-----+
|    2| qqqqqqqq| 0.12| 0.12| 0.12|
|    2|aUniqueID| 0.03| 0.03| 0.03|
|    2| qqqqqqqq|-3.84|-3.84|-3.84|
|    2| 12345678| -0.1| -0.1| -0.1|
|    2| 12345678|-0.15|-0.15|-0.15|
|    2| 12345678| 0.47| 0.47| 0.47|
|    2| 12345678|-0.06|-0.06|-0.06|
|    2| 12345678|-0.09|-0.09|-0.09|
|    2| 12345678| 0.21| 0.21| 0.21|
|    2| 12345678|-0.08|-0.08|-0.08|
|    2| 12345678| 0.44| 0.44| 0.44|
|    2|    gholi| 0.76| 0.76| 0.76|
|    2|    gholi| 1.62| 1.62| 1.62|
|    2|    gholi| 5.81| 5.81| 5.81|
|    2| bcbcbcbc| 0.58| 0.58| 0.58|
|    2| bcbcbcbc|-8.24|-8.24|-8.24|
|    2| bcbcbcbc|-0.45|-0.45|-0.45|
|    2| bcbcbcbc| 1.03| 1.03| 1.03|
|    2|aUniqueID|-0.05|-0.05|-0.05|
|    2| qqqqqqqq|-0.44|-0.44|-0.44|
+-----+---------+-----+-----+-----+
only showing top 20 rows



In [ ]:
unique_names = df.select("CLASS").distinct()
unique_names.show()

+-----+
|CLASS|
+-----+
|    0|
|    1|
|    2|
+-----+



In [ ]:
df.createOrReplaceTempView("df")

In [ ]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SQLContext, SparkSession
from pyspark.sql.types import StructType, StructField, DoubleType, IntegerType, StringType
#sc = SparkContext.getOrCreate(SparkConf().setMaster("local[*]"))

In [ ]:
sc = SparkContext.getOrCreate(SparkConf().setMaster("local[*]"))

In [ ]:
from pyspark.sql.functions import monotonically_increasing_id
from systemds.context import SystemDSContext
import numpy as np
import pandas as pd

In [ ]:
def dft_systemds(signal,name):


    with SystemDSContext(spark) as sds:
        size = signal.count()
        signal = sds.from_numpy(signal.toPandas().to_numpy())
        pi = sds.scalar(3.141592654)

        n = sds.seq(0,size-1)
        k = sds.seq(0,size-1)

        M = (n @ (k.t())) * (2*pi/size)

        Xa = M.cos() @ signal
        Xb = M.sin() @ signal

        index = (list(map(lambda x: [x],np.array(range(0, size, 1)))))
        DFT = np.hstack((index,Xa.cbind(Xb).compute()))
        DFT_pdf = pd.DataFrame(DFT, columns=list(["id",name+'_sin',name+'_cos']))
        DFT_df = spark.createDataFrame(DFT_pdf)
        return DFT_df

In [ ]:
x0 = spark.sql("SELECT X from df where CLASS = 1") ###YOUR_CODE_GOES_HERE### => Please create a DataFrame containing only measurements of class 0 from the x axis
y0 = spark.sql("SELECT Y from df where CLASS = 1") ###YOUR_CODE_GOES_HERE### => Please create a DataFrame containing only measurements of class 0 from the y axis
z0 = spark.sql("SELECT Z from df where CLASS = 1") ###YOUR_CODE_GOES_HERE### => Please create a DataFrame containing only measurements of class 0 from the z axis
x1 = spark.sql("SELECT X from df where CLASS = 2") ###YOUR_CODE_GOES_HERE### => Please create a DataFrame containing only measurements of class 1 from the x axis
y1 = spark.sql("SELECT Y from df where CLASS = 2") ###YOUR_CODE_GOES_HERE### => Please create a DataFrame containing only measurements of class 1 from the y axis
z1 = spark.sql("SELECT Z from df where CLASS = 2") ###YOUR_CODE_GOES_HERE### => Please create a DataFrame contain

In [ ]:
x0.show()

+-----+
|    X|
+-----+
|  0.0|
|  0.0|
|  0.0|
|  0.0|
|  0.0|
| 0.01|
|-0.01|
|-0.01|
|  0.0|
|-0.01|
|-0.01|
|  0.0|
|  0.0|
|-0.01|
|  0.0|
| 0.01|
|-0.01|
|-0.01|
| 0.01|
|  0.0|
+-----+
only showing top 20 rows



In [ ]:
from pyspark.sql.functions import lit

df_class_0 = dft_systemds(x0,'x') \
    .join(dft_systemds(y0,'y'), on=['id'], how='inner') \
    .join(dft_systemds(z0,'z'), on=['id'], how='inner') \
    .withColumn('class', lit(0))

df_class_1 = dft_systemds(x1,'x') \
    .join(dft_systemds(y1,'y'), on=['id'], how='inner') \
    .join(dft_systemds(z1,'z'), on=['id'], how='inner') \
    .withColumn('class', lit(1))

df_dft = df_class_0.union(df_class_1)

df_dft.show()

/usr/local/lib/python3.10/dist-packages/pyspark/sql/pandas/conversion.py:371: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():
/usr/local/lib/python3.10/dist-packages/pyspark/sql/pandas/conversion.py:371: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():
/usr/local/lib/python3.10/dist-packages/pyspark/sql/pandas/conversion.py:371: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():
/usr/local/lib/python3.10/dist-packages/pyspark/sql/pandas/conversion.py:371: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():
/usr/local/lib/python3.10/dist-packages/pyspark/sql/pandas/conversion.py:371: FutureWarning: iteritems is deprecated and wil

+----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-----+
|  id|               x_sin|               x_cos|               y_sin|               y_cos|               z_sin|               z_cos|class|
+----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-----+
| 8.0|0.007264897147586408|0.050779226648936575|0.007264897147586408|0.050779226648936575|0.007264897147586408|0.050779226648936575|    0|
| 0.0|               -0.13|                 0.0|               -0.13|                 0.0|               -0.13|                 0.0|    0|
| 7.0| 0.04362147198699657| 0.04661273969005281| 0.04362147198699657| 0.04661273969005281| 0.04362147198699657| 0.04661273969005281|    0|
|49.0|-0.07768949082949561|0.039133460895471164|-0.07768949082949561|0.039133460895471164|-0.07768949082949561|0.039133460895471164|    0|
|29.0|-0.10895275079055323|

In [ ]:
from pyspark.ml.feature import VectorAssembler

In [ ]:
vectorAssembler = VectorAssembler(inputCols=['x_sin','x_cos','y_sin','y_cos','z_sin','z_cos'], outputCol="features") ###YOUR_CODE_GOES_HERE###

In [ ]:
from pyspark.ml.classification import GBTClassifier
classifier = GBTClassifier(labelCol='class', featuresCol='features', maxIter=10) ###YOUR_CODE_GOES_HERE###

In [ ]:
from pyspark.ml import Pipeline
pipeline = Pipeline(stages=[vectorAssembler, classifier])

In [ ]:
model = pipeline.fit(df_dft)

In [ ]:
prediction = model.transform(df_dft)

In [ ]:
prediction.show()

+----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-----+--------------------+--------------------+--------------------+----------+
|  id|               x_sin|               x_cos|               y_sin|               y_cos|               z_sin|               z_cos|class|            features|       rawPrediction|         probability|prediction|
+----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-----+--------------------+--------------------+--------------------+----------+
| 8.0|0.007264897147586408|0.050779226648936575|0.007264897147586408|0.050779226648936575|0.007264897147586408|0.050779226648936575|    0|[0.00726489714758...|[0.88066422361181...|[0.85337596077307...|       0.0|
| 0.0|               -0.13|                 0.0|               -0.13|                 0.0|               -0.13|                 0.0|    0|[-0.13,0.0

In [ ]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
binEval = MulticlassClassificationEvaluator().setMetricName("accuracy") .setPredictionCol("prediction").setLabelCol("class")

binEval.evaluate(prediction)

0.9981761070017225

In [ ]:
!rm -Rf a2_m4.json

In [ ]:
prediction = prediction.repartition(1)
prediction.write.json('a2_m4.json')

In [ ]:
!rm -f rklib.py
!wget wget https://raw.githubusercontent.com/IBM/coursera/master/rklib.py

--2023-06-27 09:08:15--  http://wget/
Resolving wget (wget)... failed: Name or service not known.
wget: unable to resolve host address ‘wget’
--2023-06-27 09:08:15--  https://raw.githubusercontent.com/IBM/coursera/master/rklib.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2540 (2.5K) [text/plain]
Saving to: ‘rklib.py’

rklib.py            100%[===================>]   2.48K  --.-KB/s    in 0s      

2023-06-27 09:08:15 (29.3 MB/s) - ‘rklib.py’ saved [2540/2540]

FINISHED --2023-06-27 09:08:15--
Total wall clock time: 0.3s
Downloaded: 1 files, 2.5K in 0s (29.3 MB/s)


In [ ]:
from rklib import zipit
zipit('a2_m4.json.zip','a2_m4.json')

In [ ]:
!base64 a2_m4.json.zip > a2_m4.json.zip.base64

In [ ]:
from rklib import submit
key = "-fBiYHYDEeiR4QqiFhAvkA"
part = "IjtJk"
email = "Sowmyaag@skillup.online" ###YOUR_CODE_GOES_HERE###
submission_token = "CchQx5U34dXYAwhA" ###YOUR_CODE_GOES_HERE### # (have a look here if you need more information on how to obtain the token https://youtu.be/GcDo0Rwe06U?t=276)

with open('a2_m4.json.zip.base64', 'r') as myfile:
    data=myfile.read()
submit(email, submission_token, key, part, [part], data)

Submission successful, please check on the coursera grader page for the status
-------------------------
{"elements":[{"itemId":"B8wXV","id":"f_F-qCtuEei_fRLwaVDk3g~B8wXV~ZYDAQhTKEe6cUQp88G_pYQ","courseId":"f_F-qCtuEei_fRLwaVDk3g"}],"paging":{},"linked":{}}
-------------------------
